In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 9912422/9912422 [00:04<00:00, 2071666.07it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 71599.59it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2469493.77it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4541246.43it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [4]:
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [10]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accutacy:{correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n', end='\r')

In [12]:
n_epoch = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(n_epoch):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(), 'models/mnist_mlp_model{}.pth'.format(n_epoch))

Train Epoch: 1 [0/60000(0%)]	Loss: 1.550501
Train Epoch: 1 [640/60000(1%)]	Loss: 1.505362
Train Epoch: 1 [1280/60000(2%)]	Loss: 1.531405
Train Epoch: 1 [1920/60000(3%)]	Loss: 1.555908
Train Epoch: 1 [2560/60000(4%)]	Loss: 1.530517
Train Epoch: 1 [3200/60000(5%)]	Loss: 1.575528
Train Epoch: 1 [3840/60000(6%)]	Loss: 1.582008
Train Epoch: 1 [4480/60000(7%)]	Loss: 1.506778
Train Epoch: 1 [5120/60000(9%)]	Loss: 1.532286
Train Epoch: 1 [5760/60000(10%)]	Loss: 1.547249
Train Epoch: 1 [6400/60000(11%)]	Loss: 1.521013
Train Epoch: 1 [7040/60000(12%)]	Loss: 1.607249
Train Epoch: 1 [7680/60000(13%)]	Loss: 1.513012
Train Epoch: 1 [8320/60000(14%)]	Loss: 1.527369
Train Epoch: 1 [8960/60000(15%)]	Loss: 1.503687
Train Epoch: 1 [9600/60000(16%)]	Loss: 1.542033
Train Epoch: 1 [10240/60000(17%)]	Loss: 1.528682
Train Epoch: 1 [10880/60000(18%)]	Loss: 1.522545
Train Epoch: 1 [11520/60000(19%)]	Loss: 1.491420
Train Epoch: 1 [12160/60000(20%)]	Loss: 1.525229
Train Epoch: 1 [12800/60000(21%)]	Loss: 1.496896
